In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import gc
import time
from pandas.core.common import SettingWithCopyWarning
import warnings
from sklearn.model_selection import GroupKFold, GridSearchCV

from sklearn.preprocessing import OneHotEncoder

# I don't like SettingWithCopyWarnings ...
warnings.simplefilter('error', SettingWithCopyWarning)
gc.enable()
%matplotlib inline

In [2]:
train = pd.read_csv('train.csv')
train1 = pd.read_csv('train1.csv')
test = pd.read_csv('test.csv')
train = pd.concat([train,train1])
train.head()

/Users/maqianli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,alpha,flip_y,id,l1_ratio,max_iter,n_classes,n_clusters_per_class,n_features,n_informative,n_jobs,n_samples,penalty,random_state,scale,time
0,0.0001,0.074798,0.0,0.304083,417,4,3,327,7,-1,1089,none,475,24.242009,0.409987
1,0.0010,0.077781,1.0,0.727744,578,4,5,373,7,1,790,l1,569,54.626302,3.950953
2,0.0100,0.030196,2.0,0.745885,588,2,5,1198,6,2,428,none,529,17.999964,0.368702
3,0.0010,0.057261,3.0,0.474605,829,6,5,313,7,4,877,none,103,82.257222,1.004559
4,0.0010,0.073728,4.0,0.395049,167,8,5,644,11,2,216,elasticnet,418,95.515601,0.802800


In [3]:
tmp = train['n_jobs']
tmp = tmp.replace({-1:16})
train['n_jobs'] = tmp

tmp = test['n_jobs']
tmp = tmp.replace({-1:16})
test['n_jobs'] = tmp

#train = train.drop(38,axis = 0)
#train.head(40)

In [4]:
train['penalty_none'] = train['penalty'] == 'none'
train['penalty_l1'] = train['penalty'] == 'l1'
train['penalty_l2'] = train['penalty'] == 'l2'
train['penalty_elasticnet'] = train['penalty'] == 'elasticnet'

test['penalty_none'] = test['penalty'] == 'none'
test['penalty_l1'] = test['penalty'] == 'l1'
test['penalty_l2'] = test['penalty'] == 'l2'
test['penalty_elasticnet'] = test['penalty'] == 'elasticnet'
train['penalty_none'] = train['penalty_none'].astype('int')
train['penalty_l1'] = train['penalty_l1'].astype('int')
train['penalty_l2'] = train['penalty_l2'].astype('int')
train['penalty_elasticnet'] = train['penalty_elasticnet'].astype('int')

test['penalty_none'] = test['penalty_none'].astype('int')
test['penalty_l1'] = test['penalty_l1'].astype('int')
test['penalty_l2'] = test['penalty_l2'].astype('int')
test['penalty_elasticnet'] = test['penalty_elasticnet'].astype('int')
train['l1_new'] = 1.0 / train['l1_ratio']
test['l1_new'] = 1.0 / test['l1_ratio']
train['n_jobs_new'] = 1.0 / train['n_jobs']
test['n_jobs_new'] = 1.0 / test['n_jobs']
#train['alpha_new'] = np.log(train['alpha'])
train['clusters'] = train['n_classes']*train['n_clusters_per_class']
test['clusters'] = test['n_classes']*test['n_clusters_per_class']
train['features_new'] = train['n_features'] - train['n_informative']
test['features_new'] = test['n_features'] - test['n_informative']
train['flip'] = train['n_samples']*train['flip_y']
test['flip'] = test['n_samples']*test['flip_y']
train['l1'] = train['l1_ratio']*train['penalty_l1']
test['l1'] = test['l1_ratio']*test['penalty_l1']
train.corrwith(train.time)

alpha                   0.005835
flip_y                  0.118694
id                      0.050309
l1_ratio               -0.005580
max_iter                0.266155
n_classes               0.168375
n_clusters_per_class   -0.066400
n_features              0.227242
n_informative           0.117483
n_jobs                 -0.252937
n_samples               0.311411
random_state            0.037263
scale                   0.002713
time                    1.000000
penalty_none           -0.219776
penalty_l1              0.179525
penalty_l2             -0.220143
penalty_elasticnet      0.253320
l1_new                 -0.018055
n_jobs_new              0.359486
clusters                0.082313
features_new            0.226733
flip                    0.295643
l1                      0.150894
dtype: float64

In [5]:
#train_features = ['flip','features_new','max_iter','n_jobs','n_jobs_new','clusters',
#       'n_samples', 'n_features', 'n_classes','n_informative', 'flip_y','n_clusters_per_class',
#                  'penalty_elasticnet','penalty_l1','penalty_l2','penalty_none']
train_features = ['flip_y','max_iter', 'n_classes',
       'n_features', 'n_informative', 'n_jobs',
       'n_samples','penalty_none','penalty_l1', 'penalty_l2', 'penalty_elasticnet', 
        'n_jobs_new', 'clusters', 'features_new', 'flip', 'l1']
y = train['time']
train.describe()

,alpha,flip_y,id,l1_ratio,max_iter,n_classes,n_clusters_per_class,n_features,n_informative,n_jobs,...,penalty_none,penalty_l1,penalty_l2,penalty_elasticnet,l1_new,n_jobs_new,clusters,features_new,flip,l1
count,440.000000,440.000000,400.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,...,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000
mean,0.003692,0.049046,199.500000,0.497356,552.402273,5.861364,3.440909,743.052273,8.686364,6.054545,...,0.263636,0.236364,0.227273,0.272727,2.762869,0.380966,20.170455,734.365909,36.446325,0.122100
std,0.004466,0.030197,115.614301,0.232223,265.749457,2.515801,1.099248,371.849891,1.530791,5.363908,...,0.441106,0.425332,0.419547,0.445869,1.906179,0.329101,11.331836,371.890455,31.496944,0.251196
min,0.000100,0.000047,0.000000,0.100356,103.000000,2.000000,2.000000,110.000000,5.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.117044,0.062500,4.000000,102.000000,0.018097,0.000000
25%,0.000100,0.021007,99.750000,0.299859,325.000000,4.000000,3.000000,428.500000,8.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,1.430858,0.125000,12.000000,419.250000,10.663187,0.000000
50%,0.001000,0.048879,199.500000,0.483407,525.000000,6.000000,3.000000,734.500000,9.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,2.068651,0.250000,18.000000,726.000000,25.935755,0.000000
75%,0.010000,0.075089,299.250000,0.698882,783.250000,8.000000,4.000000,1063.250000,10.000000,8.000000,...,1.000000,0.000000,0.000000,1.000000,3.334943,0.500000,27.000000,1054.250000,55.746554,0.000000
max,0.010000,0.099866,399.000000,0.895220,998.000000,10.000000,5.000000,1397.000000,12.000000,16.000000,...,1.000000,1.000000,1.000000,1.000000,9.964549,1.000000,50.000000,1389.000000,136.086799,0.892994


In [6]:
#
from sklearn import preprocessing
n = 440
for column in train_features:
    z = preprocessing.scale(list(train[column]) + list(test[column]))
    train[column],test[column] = z[:n],z[n:]

In [10]:

from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
kf = KFold(n_splits = 5)
oof_reg_preds_5 = np.zeros(train.shape[0])
sub_reg_preds_5 = np.zeros(test.shape[0])
tmp_train = np.zeros((train.shape[0],18))
tmp_test = np.zeros((test.shape[0],18))

y1 = np.log(y)
for i in [40,45,50,55,60,65,70,75,80,85,90,100,105,110,115,120,125]:
    oof_reg_preds_5 = np.zeros(train.shape[0])
    sub_reg_preds_5 = np.zeros(test.shape[0])
    for train_index, test_index in kf.split(train):
        X_train, X_test = train[train_features].iloc[train_index], train[train_features].iloc[test_index]
        y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
        clf = MLPRegressor(hidden_layer_sizes = (70,i,65,105),batch_size = 10)
        clf.fit(X_train,y_train)
        oof_reg_preds_5[test_index] = clf.predict(X_test)

        _preds = clf.predict(test[train_features])
        sub_reg_preds_5 += _preds / 5

    sub_reg_preds_5 = np.exp(sub_reg_preds_5)
    print(i,mean_squared_error(y, np.exp(oof_reg_preds_5)))
    tmp_train[:,(i-40)//5] = np.exp(oof_reg_preds_5)
    tmp_test[:,(i-40)//5] = sub_reg_preds_5 


40 2.5746297995914893
45 2.3865795138981887
50 2.673120850790572
55 2.244895417495647
60 1.787139170404597
65 2.427249948034735
70 3.125635416720773
75 2.4147727534208205
80 3.32152345173032
85 2.472149550386992
90 2.7050531262415465
100 1.895485227115107
105 1.8695396696074724
110 1.6739889182411534
115 2.727784393270595
120 3.752973914492515
125 2.020143872698587


In [12]:
tmp_train = pd.DataFrame(tmp_train)
tmp_test = pd.DataFrame(tmp_test)
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit (tmp_train,train['time'])             
reg.predict(tmp_test)


array([ 1.31272477,  9.81671845,  0.31359228,  1.61432183,  2.81346615,
        9.11719131,  2.70662435,  0.43958667, 10.68251012,  0.8922959 ,
        5.30196592, 15.43834666,  0.80010126, 32.45154244,  0.27861231,
        0.64175933,  0.57314689,  8.00796272,  4.19584477,  1.37852341,
        0.23320131,  0.69642754,  0.44252156,  0.66606936,  1.15619167,
        2.14022027,  1.86178382,  3.19825085,  1.84310662,  2.01810743,
       17.67097385,  2.79492872,  0.4674783 ,  6.37297971,  7.15790282,
        0.66994621,  3.39084384,  2.39112687,  0.94040966,  1.81982109,
        4.74569009,  9.74596222,  1.1462352 ,  5.41961504,  0.59702074,
        3.92210869,  3.96671031,  0.79407803,  7.45879362,  0.53141048,
        0.3378024 , 18.98564981,  0.09078987,  6.1773188 , 15.68274165,
        0.19931307,  1.77674491,  0.85218088,  0.68170586,  0.77806511,
        1.77892169,  6.63683647,  0.50746837,  9.22062096,  2.86499544,
       11.24923722,  1.06682624,  0.60581487,  9.86073549,  1.78

In [13]:
#z = (sub_reg_preds_1 + sub_reg_preds_2 + sub_reg_preds_5) / 3
#(1,2,3,4)->11,5->13
z = sub_reg_preds_5
z = reg.predict(tmp_test)
ans = pd.DataFrame()
ans['Id'] = test['id']
ans['time']  = z
ans.to_csv('myresult.csv',index = 0)

In [170]:
#随机森林
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10)
oof_reg_preds_1 = np.zeros(train.shape[0])
sub_reg_preds_1 = np.zeros(test.shape[0])
count = 1
y1 = np.log(y)
for train_index, test_index in kf.split(train):
    print(count)
    count += 1
    X_train, X_test = train[train_features].iloc[train_index], train[train_features].iloc[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    clf = RandomForestRegressor(n_estimators=10,bootstrap=True,min_samples_split=4)
    clf.fit(X_train,y_train)

    oof_reg_preds_1[test_index] = clf.predict(X_test)
    _preds = clf.predict(test[train_features])
    sub_reg_preds_1 += _preds / 10

sub_reg_preds_1 = np.exp(sub_reg_preds_1)
mean_squared_error(y,np.exp(oof_reg_preds_1))

1
2
3
4
5
6
7
8
9
10


11.587562500066948

In [173]:
#svm
from sklearn import svm
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10)
oof_reg_preds_2 = np.zeros(train.shape[0])
sub_reg_preds_2 = np.zeros(test.shape[0])
count = 1
y1 = np.log(y)
for train_index, test_index in kf.split(train):
    count += 1
    X_train, X_test = train[train_features].iloc[train_index], train[train_features].iloc[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    clf = svm.SVR(kernel = 'linear',C=100)

    clf.fit(X_train,y_train)

    oof_reg_preds_2[test_index] = clf.predict(X_test)

    _preds = clf.predict(test[train_features])
    sub_reg_preds_2 += _preds / 10
sub_reg_preds_2 = np.exp(sub_reg_preds_2)
mean_squared_error(y, np.exp(oof_reg_preds_2))

3.2035109285259677

In [171]:
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.neighbors.nearest_centroid import NearestCentroid
kf = KFold(n_splits = 10)
oof_reg_preds_3 = np.zeros(train.shape[0])
sub_reg_preds_3 = np.zeros(test.shape[0])

y1 = np.log(y)
for train_index, test_index in kf.split(train):
    X_train, X_test = train[train_features].iloc[train_index], train[train_features].iloc[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    clf = tree.DecisionTreeRegressor(max_depth = 9)
    clf.fit(X_train,y_train)
    oof_reg_preds_3[test_index] = clf.predict(X_test)

    _preds = clf.predict(test[train_features])
    sub_reg_preds_3 += _preds / 10
sub_reg_preds_3 = np.exp(sub_reg_preds_3)
mean_squared_error(y, np.exp(oof_reg_preds_3))

13.840984066003623

In [172]:
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.neighbors.nearest_centroid import NearestCentroid
kf = KFold(n_splits = 10)
oof_reg_preds_4 = np.zeros(train.shape[0])
sub_reg_preds_4 = np.zeros(test.shape[0])

y1 = np.log(y)
for train_index, test_index in kf.split(train):
    X_train, X_test = train[train_features].iloc[train_index], train[train_features].iloc[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    clf = linear_model.Ridge(alpha = 10)
    clf.fit(X_train,y_train)
    oof_reg_preds_4[test_index] = clf.predict(X_test)

    _preds = clf.predict(test[train_features])
    sub_reg_preds_4 += _preds / 10
sub_reg_preds_4 = np.exp(sub_reg_preds_4)
mean_squared_error(y, np.exp(oof_reg_preds_4))

7.965127925930694

In [10]:
#import tensorflow as tf
#from sklearn.model_selection import train_test_split
#from sklearn.model_selection import KFold



kf = KFold(n_splits = 5)
oof_reg_preds_5 = np.zeros(train.shape[0])
sub_reg_preds_5 = np.zeros(test.shape[0])

y1 = np.log(y)

for train_index, test_index in kf.split(train):
    X_train, X_test = train[train_features].iloc[train_index], train[train_features].iloc[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
    regressor = tf.contrib.learn.DNNRegressor(hidden_units=[100,100,100],dropout=0.1,feature_columns=feature_columns,
                                              optimizer=tf.train.ProximalAdagradOptimizer(
                                                      learning_rate=0.1,l1_regularization_strength=0.01,l2_regularization_strength=0.01))
    regressor.fit(x=X_train,y=y_train,steps=40000)


    oof_reg_preds_5[test_index] = np.array(list(regressor.predict(X_test)))

    _preds = np.array(list(regressor.predict(test[train_features])))
    sub_reg_preds_5 += _preds / 5

sub_reg_preds_5 = np.exp(sub_reg_preds_5)
mean_squared_error(y, np.exp(oof_reg_preds_5))



#print('ev: {}'.format(ev))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c3fadc0f0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/lh/k8m8t4s17vschr0jyhjm_gb00000gn/T/tmplr2ozbgd'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving 

INFO:tensorflow:global_step/sec: 209.914
INFO:tensorflow:loss = 0.15777123, step = 6801 (0.476 sec)
INFO:tensorflow:global_step/sec: 270.442
INFO:tensorflow:loss = 0.19754961, step = 6901 (0.370 sec)
INFO:tensorflow:global_step/sec: 263.687
INFO:tensorflow:loss = 0.2663383, step = 7001 (0.379 sec)
INFO:tensorflow:global_step/sec: 270.417
INFO:tensorflow:loss = 0.22233668, step = 7101 (0.370 sec)
INFO:tensorflow:global_step/sec: 269.112
INFO:tensorflow:loss = 0.16397855, step = 7201 (0.372 sec)
INFO:tensorflow:global_step/sec: 202.294
INFO:tensorflow:loss = 0.14356697, step = 7301 (0.494 sec)
INFO:tensorflow:global_step/sec: 269.085
INFO:tensorflow:loss = 0.1173714, step = 7401 (0.372 sec)
INFO:tensorflow:global_step/sec: 269.519
INFO:tensorflow:loss = 0.17837647, step = 7501 (0.371 sec)
INFO:tensorflow:global_step/sec: 265.261
INFO:tensorflow:loss = 0.15745376, step = 7601 (0.377 sec)
INFO:tensorflow:global_step/sec: 267.993
INFO:tensorflow:loss = 0.3160092, step = 7701 (0.373 sec)
INF

INFO:tensorflow:global_step/sec: 268.749
INFO:tensorflow:loss = 0.18315618, step = 15001 (0.372 sec)
INFO:tensorflow:global_step/sec: 204.667
INFO:tensorflow:loss = 0.2606166, step = 15101 (0.489 sec)
INFO:tensorflow:global_step/sec: 269.057
INFO:tensorflow:loss = 0.16347118, step = 15201 (0.372 sec)
INFO:tensorflow:global_step/sec: 268.006
INFO:tensorflow:loss = 0.40363914, step = 15301 (0.373 sec)
INFO:tensorflow:global_step/sec: 268.47
INFO:tensorflow:loss = 0.3286489, step = 15401 (0.372 sec)
INFO:tensorflow:global_step/sec: 268.688
INFO:tensorflow:loss = 0.17672841, step = 15501 (0.372 sec)
INFO:tensorflow:global_step/sec: 267.018
INFO:tensorflow:loss = 0.16202271, step = 15601 (0.375 sec)
INFO:tensorflow:global_step/sec: 218.53
INFO:tensorflow:loss = 0.22261608, step = 15701 (0.458 sec)
INFO:tensorflow:global_step/sec: 267.561
INFO:tensorflow:loss = 0.3393299, step = 15801 (0.374 sec)
INFO:tensorflow:global_step/sec: 266.78
INFO:tensorflow:loss = 0.23603328, step = 15901 (0.375 s

INFO:tensorflow:global_step/sec: 264.174
INFO:tensorflow:loss = 0.21800134, step = 23201 (0.379 sec)
INFO:tensorflow:global_step/sec: 263.717
INFO:tensorflow:loss = 0.42310962, step = 23301 (0.379 sec)
INFO:tensorflow:global_step/sec: 242.074
INFO:tensorflow:loss = 0.2892812, step = 23401 (0.414 sec)
INFO:tensorflow:global_step/sec: 236.046
INFO:tensorflow:loss = 0.15869868, step = 23501 (0.423 sec)
INFO:tensorflow:global_step/sec: 264.35
INFO:tensorflow:loss = 0.21035433, step = 23601 (0.378 sec)
INFO:tensorflow:global_step/sec: 264.255
INFO:tensorflow:loss = 0.17746837, step = 23701 (0.379 sec)
INFO:tensorflow:global_step/sec: 266.655
INFO:tensorflow:loss = 0.353223, step = 23801 (0.375 sec)
INFO:tensorflow:global_step/sec: 263.596
INFO:tensorflow:loss = 0.150074, step = 23901 (0.379 sec)
INFO:tensorflow:global_step/sec: 193.271
INFO:tensorflow:loss = 0.31324765, step = 24001 (0.517 sec)
INFO:tensorflow:global_step/sec: 246.75
INFO:tensorflow:loss = 0.34580225, step = 24101 (0.405 se

INFO:tensorflow:global_step/sec: 269.346
INFO:tensorflow:loss = 0.318119, step = 31401 (0.371 sec)
INFO:tensorflow:global_step/sec: 271.285
INFO:tensorflow:loss = 0.12953192, step = 31501 (0.369 sec)
INFO:tensorflow:global_step/sec: 268.65
INFO:tensorflow:loss = 0.15975, step = 31601 (0.372 sec)
INFO:tensorflow:global_step/sec: 256.507
INFO:tensorflow:loss = 0.35753924, step = 31701 (0.391 sec)
INFO:tensorflow:global_step/sec: 199.464
INFO:tensorflow:loss = 0.29731092, step = 31801 (0.500 sec)
INFO:tensorflow:global_step/sec: 274.51
INFO:tensorflow:loss = 0.25709075, step = 31901 (0.365 sec)
INFO:tensorflow:global_step/sec: 261.484
INFO:tensorflow:loss = 0.4035228, step = 32001 (0.382 sec)
INFO:tensorflow:global_step/sec: 273.664
INFO:tensorflow:loss = 0.13084953, step = 32101 (0.365 sec)
INFO:tensorflow:global_step/sec: 274.097
INFO:tensorflow:loss = 0.24802928, step = 32201 (0.365 sec)
INFO:tensorflow:global_step/sec: 272.387
INFO:tensorflow:loss = 0.15016298, step = 32301 (0.368 sec

...

INFO:tensorflow:loss = 0.15952739, step = 9601 (0.489 sec)
INFO:tensorflow:global_step/sec: 258.084
INFO:tensorflow:loss = 0.2233299, step = 9701 (0.387 sec)
INFO:tensorflow:global_step/sec: 266.965
INFO:tensorflow:loss = 0.1807449, step = 9801 (0.375 sec)
INFO:tensorflow:global_step/sec: 262.343
INFO:tensorflow:loss = 0.13621561, step = 9901 (0.381 sec)
INFO:tensorflow:global_step/sec: 266.947
INFO:tensorflow:loss = 0.2169728, step = 10001 (0.375 sec)
INFO:tensorflow:global_step/sec: 209.98
INFO:tensorflow:loss = 0.18762176, step = 10101 (0.476 sec)
INFO:tensorflow:global_step/sec: 266.059
INFO:tensorflow:loss = 0.29023373, step = 10201 (0.376 sec)
INFO:tensorflow:global_step/sec: 262.717
INFO:tensorflow:loss = 0.24711117, step = 10301 (0.381 sec)
INFO:tensorflow:global_step/sec: 264.805
INFO:tensorflow:loss = 0.28069416, step = 10401 (0.378 sec)
INFO:tensorflow:global_step/sec: 268.616
INFO:tensorflow:loss = 0.23327065, step = 10501 (0.372 sec)
INFO:tensorflow:global_step/sec: 206.15

INFO:tensorflow:global_step/sec: 261.781
INFO:tensorflow:loss = 0.28180617, step = 17801 (0.382 sec)
INFO:tensorflow:global_step/sec: 206.372
INFO:tensorflow:loss = 0.2660461, step = 17901 (0.485 sec)
INFO:tensorflow:global_step/sec: 265.644
INFO:tensorflow:loss = 0.22576919, step = 18001 (0.376 sec)
INFO:tensorflow:global_step/sec: 261.802
INFO:tensorflow:loss = 0.31152436, step = 18101 (0.383 sec)
INFO:tensorflow:global_step/sec: 265.758
INFO:tensorflow:loss = 0.17954919, step = 18201 (0.376 sec)
INFO:tensorflow:global_step/sec: 266.835
INFO:tensorflow:loss = 0.10188274, step = 18301 (0.375 sec)
INFO:tensorflow:global_step/sec: 209.755
INFO:tensorflow:loss = 0.19751064, step = 18401 (0.476 sec)
INFO:tensorflow:global_step/sec: 263.202
INFO:tensorflow:loss = 0.31127915, step = 18501 (0.380 sec)
INFO:tensorflow:global_step/sec: 265.707
INFO:tensorflow:loss = 0.2285953, step = 18601 (0.376 sec)
INFO:tensorflow:global_step/sec: 263.24
INFO:tensorflow:loss = 0.245841, step = 18701 (0.380 

INFO:tensorflow:loss = 0.2458419, step = 25901 (0.393 sec)
INFO:tensorflow:global_step/sec: 214.345
INFO:tensorflow:loss = 0.1783928, step = 26001 (0.467 sec)
INFO:tensorflow:global_step/sec: 239.227
INFO:tensorflow:loss = 0.1775817, step = 26101 (0.418 sec)
INFO:tensorflow:global_step/sec: 265.787
INFO:tensorflow:loss = 0.28469163, step = 26201 (0.376 sec)
INFO:tensorflow:global_step/sec: 263.091
INFO:tensorflow:loss = 0.3892902, step = 26301 (0.380 sec)
INFO:tensorflow:global_step/sec: 260.478
INFO:tensorflow:loss = 0.22628827, step = 26401 (0.384 sec)
INFO:tensorflow:global_step/sec: 266.103
INFO:tensorflow:loss = 0.14783458, step = 26501 (0.376 sec)
INFO:tensorflow:global_step/sec: 205.454
INFO:tensorflow:loss = 0.17812152, step = 26601 (0.487 sec)
INFO:tensorflow:global_step/sec: 263.006
INFO:tensorflow:loss = 0.309093, step = 26701 (0.380 sec)
INFO:tensorflow:global_step/sec: 263.272
INFO:tensorflow:loss = 0.4797265, step = 26801 (0.380 sec)
INFO:tensorflow:global_step/sec: 266.6

INFO:tensorflow:global_step/sec: 257.989
INFO:tensorflow:loss = 0.13253134, step = 34101 (0.387 sec)
INFO:tensorflow:global_step/sec: 263.97
INFO:tensorflow:loss = 0.29518613, step = 34201 (0.379 sec)
INFO:tensorflow:global_step/sec: 206.192
INFO:tensorflow:loss = 0.31451285, step = 34301 (0.485 sec)
INFO:tensorflow:global_step/sec: 256.408
INFO:tensorflow:loss = 0.21619606, step = 34401 (0.390 sec)
INFO:tensorflow:global_step/sec: 265.614
INFO:tensorflow:loss = 0.21843243, step = 34501 (0.376 sec)
INFO:tensorflow:global_step/sec: 260.987
INFO:tensorflow:loss = 0.16993749, step = 34601 (0.383 sec)
INFO:tensorflow:global_step/sec: 256.117
INFO:tensorflow:loss = 0.32463154, step = 34701 (0.391 sec)
INFO:tensorflow:global_step/sec: 206.009
INFO:tensorflow:loss = 0.11334083, step = 34801 (0.485 sec)
INFO:tensorflow:global_step/sec: 265.813
INFO:tensorflow:loss = 0.36205554, step = 34901 (0.376 sec)
INFO:tensorflow:global_step/sec: 266.276
INFO:tensorflow:loss = 0.35559353, step = 35001 (0.

6.047506977718001

In [66]:
z = sub_reg_preds_5
ans = pd.DataFrame()
ans['Id'] = test['id']
ans['time']  = z
ans.to_csv('myresult.csv',index = 0)